# Metrics

In [1]:
import numpy as np
import pandas as pd
import PIL.Image as Image
import matplotlib.pyplot as plt

from utils import *

%matplotlib inline
%load_ext autoreload

In [2]:
cfg = {
'EXPERIMENT': 'filter_',
'DATASET': 'coco',
'MODEL': 'SAM',
'TARGET': 'SAM',
'SPARSITY': 50,
'MODE': ''
}
cfg['ROOT'], cfg['N'], cfg['CLASSES'] = get_dataset_info(cfg['DATASET'])

In [ ]:
df_p = pd.read_pickle(f"results/{cfg['EXPERIMENT']}{cfg['DATASET']}_prompts.pkl")
df_0 = pd.read_pickle(f"results/{cfg['EXPERIMENT']}{cfg['DATASET']}_{cfg['TARGET']}_0.pkl")
df_s = pd.read_pickle(f"results/{cfg['EXPERIMENT']}{cfg['DATASET']}_{cfg['MODEL']}_{cfg['SPARSITY']}{cfg['MODE']}.pkl")

In [ ]:
df_p.head()

In [ ]:
# Add Image Shape
if 'shape' not in df_p.keys():
    df_p = add_image_shape(df_p)
    df_p.head()
    df_p.to_pickle(f"results/{cfg['EXPERIMENT']}{cfg['DATASET']}_prompts.pkl")

In [ ]:
df_0s = get_analytics(df_0, df_s, df_p)
df_0s.to_pickle(f"analytics/{cfg['EXPERIMENT']}{cfg['DATASET']}_{cfg['MODEL']}_{cfg['SPARSITY']}{cfg['MODE']}.pkl")
df_0s.head()

In [ ]:
df_0s.hist(column='class', bins=cfg['N'])
plt.semilogy()
plt.show()

In [ ]:
df_0s.hist(column='iou')
plt.semilogy()
plt.show()
df_0s['iou'].mean()

## Batch Save Analytics

In [10]:
cfg = {
'EXPERIMENT': 'filter_',
'DATASET': 'coco',
'MODEL': 'MobileSAM',
'TARGET': 'SAM',
'SPARSITY': 0,
'MODE': ''
}
cfg['ROOT'], cfg['N'], cfg['CLASSES'] = get_dataset_info(cfg['DATASET'])

In [12]:
for s in [0]:
    cfg['SPARSITY'] = s
    save_analytics(cfg)

## Class-wise mIoU

In [ ]:
c = 2
df_c = df_0s[df_0s['class'] == c]

In [ ]:
df_c['iou'].mean()

In [ ]:
df_c.hist(column='iou')
plt.show()

## PIEs

In [ ]:
min_size = df_0s.nsmallest(25, ['mask_size_diff'])
max_size = df_0s.nlargest(25, ['mask_size_diff'])
min_score = df_0s.nsmallest(25, ['score_diff']) # not very useful
max_score = df_0s.nlargest(25, ['score_diff']) # not very useful
min_iou = df_0s.nsmallest(25, ['iou'])
max_iou = df_0s.nlargest(25, ['iou'])
min_size.head()

In [ ]:
max_iou.head()

In [ ]:
show_samples(max_size, df_0, df_s, 5)

# Test

In [ ]:
df = pd.read_pickle(f"results/test_sa1b_SAM_0.pkl")
df.head()

In [ ]:
e = df.iloc[3]

In [ ]:
e

In [ ]:
i = np.array(Image.open(cfg['ROOT'].joinpath(f"0/{e['name']}.jpg")).convert("RGB"))
i.shape

In [ ]:
plt.imshow(i)
plt.show()

In [ ]:
m = e['mask']
m.shape

In [ ]:
o = e['mask_origin']
o

In [ ]:
plt.imshow(i[o[0]:o[0]+m.shape[0], o[1]:o[1]+m.shape[1], :])
plt.imshow(m, alpha=0.4)
plt.show()

In [ ]:
plt.imshow(m)

In [ ]:
def get_full_mask(mask, origin, full_shape):
    full_mask = np.zeros(full_shape)
    full_mask[origin[0]:origin[0]+mask.shape[0], origin[1]:origin[1]+mask.shape[1]] = mask
    return full_mask


In [ ]:
plt.imshow(i)
plt.imshow(get_full_mask(m, o, i.shape[:2]), alpha=0.4)